In [45]:
#pip install pandas==1.3.0

In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
df=pd.read_csv('/content/drive/MyDrive/train.csv',parse_dates=['datetime'])
#df=pd.read_csv('/content/drive/MyDrive/interpolate.csv',parse_dates=['datetime'])
#df=pd.read_csv('/content/drive/MyDrive/pad_data.csv',parse_dates=['datetime'])



In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
df.drop_duplicates(inplace=True)

In [49]:
#pip install fedot

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66545 entries, 0 to 67135
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   datetime                     66545 non-null  datetime64[ns]
 1   Номер скважины               66545 non-null  int64         
 2   Дебит нефти                  66545 non-null  float64       
 3   Давление забойное            64185 non-null  float64       
 4   x                            66545 non-null  float64       
 5   y                            66545 non-null  float64       
 6   Объем жидкости               2874 non-null   float64       
 7   Объем нефти                  2874 non-null   float64       
 8   Активная мощность (ТМ)       61884 non-null  float64       
 9   Время работы (ТМ)            62927 non-null  float64       
 10  Газовый фактор рабочий (ТМ)  25081 non-null  float64       
 11  Давление буферное            10184 non-nu

In [51]:
df.datetime.max()

Timestamp('1992-04-26 00:00:00')

In [52]:
test=df[df.datetime>='1992-01-13']
test=test[test.datetime<='1992-04-11']
train=df[df.datetime<'1992-01-13']

In [ ]:
import pandas as pd
import numpy as np

from fedot.api.main import Fedot
from fedot.core.repository.tasks import TsForecastingParams
from tqdm import tqdm

FORECAST_HORIZON = 90


Метод для генерации единичного прогноза

In [53]:


def make_fit_predict(historical_values: pd.DataFrame,
                     forecast_horizon: int = FORECAST_HORIZON):
    """
    Используется одномерный временной ряд для обучения FEDOT модели
    Затем используется временной ряд как предыстория для формирования прогноза
    в будущее
    """
    time_series = np.array(historical_values["Дебит нефти"])

    model = Fedot(problem='ts_forecasting',
                  task_params=TsForecastingParams(forecast_length=forecast_horizon),
                  timeout=0.5, preset='fast_train', n_jobs=-1)

    # run AutoML model design in the same way
    pipeline = model.fit(features=time_series, target=time_series,
                         predefined_model='auto')
    forecast = model.predict(time_series)

    # Generate pipeline with datetime and predicted column
    date_range = pd.date_range(start='1992-01-13', freq='1D', periods=FORECAST_HORIZON)
    forecast_df = pd.DataFrame({'datetime': date_range, 'forecast': forecast})
    return forecast_df




100%|██████████| 106/106 [01:10<00:00,  1.50it/s]


Генерация прогнозов для всех скважин в выборке

In [ ]:
def launch_baseline():
    train_df = train
    wells = list(train_df["Номер скважины"].unique())

    all_forecasts = []
    with tqdm(total=len(wells)) as pbar:
        for well in wells:
            well_df = train_df[train_df["Номер скважины"] == well]

            # Make predictions with FEDOT framework
            forecats_df = make_fit_predict(well_df)
            forecats_df["Номер скважины"] = [well] * len(forecats_df)
            all_forecasts.append(forecats_df)

            pbar.update(1)

    all_forecasts = pd.concat(all_forecasts)
    all_forecasts.to_csv('baseline_forecast_train.csv', index=False)


launch_baseline()

In [54]:
predict=pd.read_csv("/content/baseline_forecast_train.csv",parse_dates=["datetime"])

In [55]:
test.drop_duplicates(subset=['Номер скважины','datetime'],inplace=True)

In [58]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_error
from tqdm import tqdm


Расчет метрики

In [56]:
def calculate_final_score(vis=False):
    """ Расчет метрики по таблице с предсказаниями алгоритма """
    df_with_metrics = pd.read_csv("/content/baseline_forecast_train.csv", parse_dates=['datetime'])
    wells = list(df_with_metrics["Номер скважины"].unique())

    train_df = train#pd.read_csv(os.path.join(get_project_path(), 'data', 'train.csv'), parse_dates=['datetime'])
    test_df = test#pd.read_csv(os.path.join(get_project_path(), 'data', 'test.csv'), parse_dates=['datetime'])
    metrics = []
    with tqdm(total=len(wells)) as pbar:
        for well in wells:
            well_forecast_df = df_with_metrics[df_with_metrics["Номер скважины"] == well]
            well_forecast_df = well_forecast_df.sort_values(by='datetime')

            well_actual_df = test_df[test_df["Номер скважины"] == well]
            well_actual_df = well_actual_df.sort_values(by='datetime')

            rmse_metric = mean_squared_error(np.array(well_actual_df['Дебит нефти']),
                                             np.array(well_forecast_df['forecast']),
                                             squared=False)
            metrics.append(rmse_metric)

            
    

            if vis:
                # Create plot
                historical_df = train_df[train_df["Номер скважины"] == well]

                plt.plot(historical_df['datetime'], historical_df['Дебит нефти'], label='Train')
                plt.plot(well_actual_df['datetime'], well_actual_df['Дебит нефти'], label='Test')
                plt.plot(well_forecast_df['datetime'], well_forecast_df['forecast'], label='Forecast')
                plt.grid()
                plt.legend()
                plt.xlabel('Дата')
                plt.ylabel('Дебит нефти')
                plt.show()
            pbar.update(1)

    metrics = np.array(metrics)
    return np.mean(metrics)


metric = calculate_final_score( vis=False)
print(f'Метрика RMSE {metric:.2f}')


100%|██████████| 106/106 [00:00<00:00, 323.09it/s]

Метрика RMSE 3.53
